In [67]:
# Imports and global tolerance
import mercury as mr
import pandas as pd
import random
import csv

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

tolerance = 1e-7

In [68]:
# User interface inputs
show_code = mr.Checkbox(value=False, label="Show source code")
app = mr.App(
    title="Loan House Calculation",
    description="Actuarial Mathematics",
    show_code=show_code.value,
    continuous_update=True
)

admin_toggle = mr.Checkbox(value=False, label="Enable Admin Mode (Simulate Random Contracts)")
_ = mr.Note(text="---")

if admin_toggle.value == True:
    num_contracts = mr.Numeric(label="Number of Random Contracts", value=10, min=1, max=1000)
else:
    age = mr.Numeric(label="Age (Years)", value=22, min=18, max=100)
    amount = mr.Numeric(label="Loan Amount", min=0, value=150000, max=999999999999)
    interest_rate = mr.Numeric(label="Yearly Interest Rate (%)", value=2.5, min=0.0, max=100.0, step=0.1)
    maturity = mr.Numeric(label="Maturity (Years)", value=25, min=1, max=50)

mortality_table = mr.Select(value = "TD88-90 (France)", choices = ["TD88-90 (France)", "TH00-02 (France)"], label = "Select Mortality Table")
currency = mr.Select(value = "EUR", choices = ["EUR", "USD", "GBP"], label = "Select Currency")

mercury.Checkbox

mercury.App

mercury.Checkbox

---

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Select

mercury.Select

In [69]:
_ = mr.Note(text="---")
_ = mr.Note("Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.")

---

Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.

In [ ]:
# Load mortality table based on selection
if mortality_table.value == "TD88-90 (France)":
    with open("TD88-90.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        mortality_data = {int(row[0]): int(row[1]) for row in reader}

if mortality_table.value == "TH00-02 (France)":
    with open("TH00-02.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        mortality_data = {int(row[0]): int(row[1]) for row in reader}

# print(f"Successfully loaded {len(mortality_data)} entries from {mortality_table.value} mortality table.")

In [70]:
# Helper functions
def annuity_factor(x, m, tech_rate):
    act_sum = 0
    for i in range(0, m):
        act_sum += nex(x, i, tech_rate)
    return act_sum

def nex(x, n, tech_rate):
    return npx(x, n) * techDF(n, tech_rate)

def npx(x ,n):
    return mortality_data[x+n] / mortality_data[x]

def techDF(n, tech_rate):
    return 1 / ((1 + tech_rate) ** n)

def n_1qx(x, n):
    return (mortality_data[x+n] - mortality_data[x+n+1]) / mortality_data[x]

In [71]:
# Core logic
def calculate_contract(p_age, p_maturity, p_interest, p_amount):
    number_of_payments = int(p_maturity * 12) # input is in years
    periodic_rate = p_interest / 12 / 100 # input is yearly + should be divided by 100
    monthly_reimbursed_amount = (p_amount * periodic_rate) / (1 - (1 + periodic_rate) ** -number_of_payments)

    '''
    print("----")
    print(f"Number of payments: {number_of_payments}")
    print(f"Periodic rate: {periodic_rate:.6f}" + "%")
    print(f"Total amount: {amount.value:,.0f} {currency.value}")
    print(f"Monthly reimbursed amount: {monthly_reimbursed_amount:.6f} {currency.value}")
    '''

    # Calculate for each month
    remaining_amount = []
    interest = []
    amortization = []

    for i in range(0, number_of_payments + 1):
        # Special cases on the 0 month
        if i == 0:
            remaining_amount.append(p_amount)
            interest.append(0)
            amortization.append(0)
            continue

        # Remaining amount (end of the year) Ck
        # Using tolerance for the last element (not perfect zero)
        calculated_value = (1 + periodic_rate) * remaining_amount[-1] - monthly_reimbursed_amount
        remaining_amount.append(0 if calculated_value < tolerance else calculated_value)

        # Interest Ik
        interest.append(remaining_amount[i-1] * periodic_rate)
        # Amortization Ak
        amortization.append(monthly_reimbursed_amount - interest[i])

    total_reimbursement_amount = sum(amortization)
    cost_of_loan = sum(interest)
    # print(f"Total reimbursement amount: {total_reimbursement_amount:.2f} {currency.value}")
    # print(f"Cost of loan: {cost_of_loan:.2f} {currency.value}")

    # Yearly payment calculations
    k = 0
    discounted_amount_in_risks = []

    for i in range(1, len(remaining_amount)):
        if i % 12 == 0:
            benefit_akprime = remaining_amount[i]
            discount_factor = (1 / (1 + p_interest / 100)) ** (k+1)
            probability = (mortality_data[p_age+k] - mortality_data[p_age+k+1]) / mortality_data[p_age]
            discounted_amount_in_risk = benefit_akprime * discount_factor * probability
            discounted_amount_in_risks.append(discounted_amount_in_risk)
            k += 1
    total_discounted_amount_in_risk = sum(discounted_amount_in_risks)
    single_premium = total_discounted_amount_in_risk

    annuity_factor_calc = annuity_factor(int(p_age), int(p_maturity), p_interest / 100)
    annual_premium = single_premium / annuity_factor_calc

    # Monthly payment calculations
    monthly_annuity_factor = annuity_factor_calc - ((12 - 1 )/( 2 * 12)) * (1 - nex(p_age, p_maturity, p_interest / 100))
    monthly_annual_premium = single_premium / monthly_annuity_factor
    monthly_premium = monthly_annual_premium / 12

    # Return all necessary calculated values for the frontend
    return single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, remaining_amount


In [72]:
# If admin simulation
if admin_toggle.value == True:
    random_contracts = []
    for _ in range(int(num_contracts.value)):
        age = random.randint(18, 90)  # Generate the age once per contract
        loan_amount = random.randint(10000, 1000000)
        interest_rate = random.uniform(0.5, 10.0)
        maturity = random.randint(1, 100 - age) # upper do not to overflow the mortality table

        single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, remaining_amount = calculate_contract(age, maturity, interest_rate, loan_amount)

        contract = {
            'age': age,
            'loan_amount': loan_amount,
            'currency': currency.value,  # Fix within one simulation
            'interest_rate': interest_rate,
            'maturity': maturity,
            'mortality_table': mortality_table.value,  # Fix within one simulation
            'single_premium': single_premium,
            'annual_premium': annual_premium
        }
        random_contracts.append(contract)

    print(random_contracts)

    df = pd.DataFrame(random_contracts)

    # Feature selection
    X = df[['age', 'loan_amount', 'interest_rate', 'maturity']]
    y_annual = df['annual_premium']
    y_monthly = df['single_premium']

    # Split the data
    X_train, X_test, y_train_annual, y_test_annual = train_test_split(X, y_annual, test_size=0.2, random_state=42)
    X_train, X_test, y_train_monthly, y_test_monthly = train_test_split(X, y_monthly, test_size=0.2, random_state=42)

    # Train the linear regression model
    model_annual = LinearRegression()
    model_annual.fit(X_train, y_train_annual)

    model_monthly = LinearRegression()
    model_monthly.fit(X_train, y_train_monthly)

    # Predictions
    y_pred_annual = model_annual.predict(X_test)
    y_pred_monthly = model_monthly.predict(X_test)

    # Evaluate the model
    mse_annual = mean_squared_error(y_test_annual, y_pred_annual)
    r2_annual = r2_score(y_test_annual, y_pred_annual)

    mse_monthly = mean_squared_error(y_test_monthly, y_pred_monthly)
    r2_monthly = r2_score(y_test_monthly, y_pred_monthly)

    print(f"Annual Premium Model: MSE={mse_annual}, R2={r2_annual}")
    print(f"Monthly Premium Model: MSE={mse_monthly}, R2={r2_monthly}")
    mr.Stop()
else:
    # Input validation
    if age and age.value + maturity.value > 100:
        mr.Markdown(text=f'#<font color="red">ERROR: With these values no house loan is available. Please check your inputs.</font>')
        mr.Stop()

In [74]:
# Stop ploting if it is simulation
if admin_toggle.value == True:
    mr.Stop()

single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, remaining_amount = calculate_contract(age.value, maturity.value, interest_rate.value, amount.value)

colors = ["#F8F9FA", "green", "#FCE700", "#00F5FF"]
mr.Markdown(text=f'#<font color="black">Yearly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{single_premium:.2f}"), title=f"Single premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annual_premium:.2f}"), title=f"Annual premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annuity_factor_calc:.3f}"), title="Annuity factor")
])

#<font color="black">Yearly Payment</font>

In [75]:
mr.Markdown(text=f'#<font color="black">Monthly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{monthly_premium:.2f}"), title=f"Monthly premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annual_premium:.2f}"), title=f"Annual premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annuity_factor:.3f}"), title="Annuity factor")
])

#<font color="black">Monthly Payment</font>

In [76]:
# Balance sheet for year 1
# Sum of Assets and Liability should be equal

# Assets
balance_sheet_annual_premium = annual_premium
balance_sheet_interest = balance_sheet_annual_premium * interest_rate.value / 100
balance_sheet_sum_of_assets = balance_sheet_annual_premium + balance_sheet_interest

# Liability
balance_sheet_claims = remaining_amount[12] * (1 - mortality_data[age.value + 1 ] / mortality_data[age.value])
balance_sheet_recurrence = (0 + balance_sheet_annual_premium - remaining_amount[12] * (1 / (1 + interest_rate.value / 100)) * n_1qx(age.value+12/12-1, 0)) / ((1 / (1 + interest_rate.value / 100)) * (1 -  n_1qx(age.value+12/12-1, 0)))
balance_sheet_reserves = balance_sheet_recurrence * mortality_data[age.value + 1 ] / mortality_data[age.value]
balance_sheet_sum_of_liabilities = balance_sheet_claims + balance_sheet_reserves

'''
print(f"Claims: {balance_sheet_claims:.2f} {currency.value}")
print(f"Recurrence: {balance_sheet_recurrence:.5f} {currency.value}")
print(f"Reserves: {balance_sheet_reserves:.4f} {currency.value}")
'''

# Validation check: assets should be equal to liabilities
if balance_sheet_sum_of_assets - balance_sheet_sum_of_liabilities > tolerance:
    mr.Markdown(text=f'#<font color="red">ERROR: The difference between the assets and the liabilities are greater than the tolerance.</font>')

mr.Markdown(text=f'#<font color="black">Balance Sheet for Year 1</font>')

df = pd.DataFrame(
    {
        "Assets": ["Annual Premium*", "Interest (financial income)", "Opening", "Sum of Assets"],
        "Assets Value": [
            f"{float(balance_sheet_annual_premium):.2f} {currency.value}",
            f"{float(balance_sheet_interest):.2f} {currency.value}",
            f"{float(0):.2f} {currency.value}",
            f"{float(balance_sheet_sum_of_assets):.2f} {currency.value}"
        ],
        "Liability": ["Claims", "Premium Reserves", "", "Sum of Liabilities"],
        "Liability Value": [
            f"{float(balance_sheet_claims):.2f} {currency.value}",
            f"{float(balance_sheet_reserves):.2f} {currency.value}",
            "",
            f"{float(balance_sheet_sum_of_liabilities):.2f} {currency.value}"
        ]
    }
)

mr.Table(data=df, width="150px", text_align="left")

#<font color="black">Balance Sheet for Year 1</font>

In [77]:
print("* We are using annual premium with yearly payment for balance sheet calculations.")

* We are using annual premium with yearly payment for balance sheet calculations.
